In [1]:
#Used in Tensorflow Model
import numpy as np
import tensorflow as tf
import tflearn
import random

#Usde to for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [4]:
with open('intents.json', 'r', encoding='utf-8') as json_data:
    intents = json.load(json_data)

In [5]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [6]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

1698 documents
241 classes ['administrasi_sidang_biaya_perkara', 'administrasi_sidang_hak_persidangan', 'administrasi_sidang_hitung_perkara', 'administrasi_sidang_panjar_habis', 'administrasi_sidang_persyaratan', 'administrasi_sidang_pertimbangan_panjar', 'aduan_layanan_prosedur', 'ecourt_batas_waktu_bayar', 'ecourt_bayar', 'ecourt_bayar_titip', 'ecourt_call_center', 'ecourt_daftar_tanpa_email', 'ecourt_data', 'ecourt_definisi', 'ecourt_dokumen_daftar_perkara', 'ecourt_e-filling', 'ecourt_e-litigation', 'ecourt_e-payment', 'ecourt_e-summons', 'ecourt_edit_dokumen', 'ecourt_email_aktivasi', 'ecourt_info_sisa_panjar', 'ecourt_info_tambahan_panjar', 'ecourt_konfirmasi_salah_bayar', 'ecourt_kredit', 'ecourt_kuasa_lebih_dari_satu', 'ecourt_layanan', 'ecourt_legal_daftar', 'ecourt_non_advokat', 'ecourt_panggilan_elektronik_biaya', 'ecourt_panggilan_elektronik_email', 'ecourt_panggilan_luar_negeri', 'ecourt_panggilan_pertama_tergugat', 'ecourt_pembayaran_bank', 'ecourt_pemula', 'ecourt_pengaj

In [7]:
training = []
output = []
output_empty = [0] * len(classes)

for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [10]:
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])
tf.compat.v1.reset_default_graph()

In [11]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Training....")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Training....


In [12]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [14]:
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')


Training Step: 216302  | total loss: 0.08662 | time: 1.186s
| Adam | epoch: 1016 | loss: 0.08662 - acc: 0.9448 -- iter: 1696/1698
Training Step: 216303  | total loss: 0.08170 | time: 1.191s
| Adam | epoch: 1016 | loss: 0.08170 - acc: 0.9503 -- iter: 1698/1698
--
INFO:tensorflow:d:\TUGAS KULIAH\SEMESTER 8\PENELITIAN CHATBOT\tensorflow-sample\AI_ChatBot_Python-master\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [29]:
print("Pickle is also Saved..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data.pkl", "wb" ) )

Pickle is also Saved..........


In [30]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data.pkl", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intents.json', encoding='utf-8') as json_data:
    intents = json.load(json_data)
    
print("Loading the Model......")
# load our saved model
model.load('./model.tflearn')

Loading Pickle.....
Loading the Model......
INFO:tensorflow:Restoring parameters from d:\TUGAS KULIAH\SEMESTER 8\PENELITIAN CHATBOT\tensorflow-sample\AI_ChatBot_Python-master\model.tflearn


In [31]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.35

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

In [32]:
while True:
    input_data = input("Tanya > ")
    answer = response(input_data)
    answer

Prosedur Berperkara Gugatan Waris
1. Gugatan waris diajukan ke Pengadilan Agama oleh penggugat selaku ahli waris dan dapat pula  mengguganakan jasa pengacara/advokat atau kuasa insidentil. Jika menggunakan kuasa insidentil, terlebih dahulu harus mengajukan permohonan  kepada Ketua Pengadilan Agama untuk menjadi kuasa insidentil, kemudian Ketua Pengadilan mengeluarkan surat izinnya.
2. Pengajuan gugatan waris disertai dengan bukti  kematian pewaris dari Lurah/Kepala Desa dan silsilah ahli warisnya dan dipersiapkan pula dokumen bukti-bukti kepemilikan objek sengketa seperti sertifikat, akta jual beli, dan bukti kepemilikan lainnya.
3. Dalam surat gugatan harus memuat secara lengkap objek-objek sengketa mengenai ukuran dan batas-batasnya tanah, merek dan tahun pembuatan dan kalau perlu dengan warnanya jika objeknya berupa mobil/Sepeda motor atau barang-barang elektronik.
4. Pengujuan gugatan waris diajukan ke Pengadilan Agama  yang daerah hukumnya meliputi letak barang tetap  (objek sengk

KeyboardInterrupt: Interrupted by user